# Beach Resources OW VisTool/STHLM office

In [1]:
## set up Dash with classes for all of the visual components of the application
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim 
from itertools import count
from datetime import datetime
import itertools
import plotly 
import plotly.plotly as py
from plotly.graph_objs import *
import os
import folium
import requests, json 
import pycountry





In [25]:
# Reading '.xlsx' file 
def import_excel(xlxs_file_nme, sheet_num,sheet_idx=False, file_loc=True):
    if file_loc:
        file_path = os.path.abspath(xlxs_file_nme)
        xlsx = pd.ExcelFile(file_path)
        sheet1 = xlsx.parse(sheet_num)
        if sheet_idx:
            sheet1.index = sheet1['Name']
            new_df_beach_0 = sheet1.iloc[:]
            return new_df_beach_0
        else:
            new_df_beach_0 = sheet1.iloc[:]
            return new_df_beach_0
    else:
        xlsx = pd.ExcelFile(xlxs_file_nme)
        sheet1 = xlsx.parse(sheet_num)
        if sheet_idx:
            sheet1.index = sheet1['Name']
            new_df_beach_0 = sheet1.iloc[:]
            return new_df_beach_0
        else:
            new_df_beach_0 = sheet1.iloc[:]
            return new_df_beach_0

new_df_beach = import_excel(
    'Copy of Beach Work HTMLversion.xlsx', sheet_num=0, sheet_idx=False
) #, sheet_idx='Name'
new_df_beach.head()


,Name,Beach,Location,Responsible,Case code,Priority,Client,Project,Deadline/End date,Days until DL,Status,Comment
NaN,Alexander,yes,Sweden,NaN,Carl - FSP31001,Internal - Firm building,Bank A,NaN,2018-04-14,80,Ongoing,NaN
NaN,Anders,yes,United Kingdom,NaN,Sean - FSP03201,Internal - Business development,Bank B,NaN,2018-04-14,80,Ongoing,NaN
NaN,Andreas,yes,Germany,NaN,Bo - N/A,Project - Soft start,Bank A,NaN,2018-04-14,80,Ongoing,NaN
NaN,Artin,no,France,NaN,Nader - FSP11101,Project - Delivery risk / WLB,Bank B,NaN,2018-04-14,80,NaN,NaN
NaN,Axel,yes,Italy,NaN,Martin - FSP32601,Project - General help,Bank A,NaN,2018-04-14,80,Ongoing,NaN


In [26]:
# Building a pivot table to plot the beach resources
#new_df_beach['Deadline/End date'].value_counts()
#
## 1D counting given one feature to DF
uniq, count = np.unique(new_df_beach['Priority'], return_counts=True)
_1D_count = dict(zip(uniq, count))

## 2D counting given 2 features to DF
df_grop = new_df_beach.groupby(['Beach','Deadline/End date', 'Name'])
#list(zip(*df_grop)).sum()
plt_df = df_grop.count()
new_plt_df = pd.DataFrame(np.array(list(zip(*df_grop.Beach))[0]), columns=['Beach','Deadline/End date', 'Name'])
#pd.to_datetime(new_plt_df[1])
new_plt_df.replace(pd.to_datetime(new_plt_df['Deadline/End date'], format='%Y%m%d'), new_df_beach['Deadline/End date'])
#plt.bar(x=plt_df['Days until DL'], y=plt_df['Deadline/End date'])
counts = new_plt_df['Deadline/End date'].value_counts()
beach = new_plt_df['Beach'].value_counts()

In [27]:
################ plotting using bokeh##############
#source = ColumnDataSource(data = dict(x=counts.index,count=counts.values))
#data = pd.DataFrame(counts)
#output_file("test.html")
##Creating plots using bokeh
#date = [datetime.strftime(x, format='%Y-%m-%d') for x in counts.index]
#source = ColumnDataSource(data=dict(x_1=date, counts=list(counts.values)))
#p = figure()
#p = figure(x_range=date,plot_height=250,toolbar_location=None, title="Beach resources Deadline")
#p.background_fill_color = "whitesmoke"
#p.background_fill_alpha = .9
#p.vbar(x='x_1',width=.4, top='counts', source=source, legend="Beach Res. count")
#p.border_fill_color = "whitesmoke"
#p.min_border_left = 10
#p.xgrid.grid_line_color = None
#p.ygrid.grid_line_alpha = 0.9
#p.ygrid.grid_line_dash = [6, 4]
#show(p)
################ plotting using matplotlib##############
## set up a plot frame 
#fig, axs = plt.subplots(1,2, figsize=(13,5))
#for ax in axs:
   # ax.tick_params(color='white', labelcolor='white')
    #for spine in ax.spines.values():
        #spine.set_edgecolor('white')
#axs[0].bar(counts.index, counts.values)

#axs[1].pie(new_df_beach['Priority'].value_counts().values, 
        #labels=new_df_beach['Priority'].value_counts().index)




In [28]:
# GeoSpotting for all resource 
def country_conversion(input_countries):
    """
    converts cuntry name to a alpha_3 nickname (e.g. 'Canada' >> 'CAN')
    Inputs:
    __________________
     - input_countries : str
        dataframe or list of country names
    Outputs:
    __________________
     list of countries nicknames 
    """
    countries = {}
    for Country in pycountry.countries:
        countries[Country.name] = Country.alpha_3
    codes = [countries.get(country, 'Unknown code') for country in input_countries]
    return codes

def poptext_2(df,location_col,name_col, x):
    """
    Supporting func to show info for each marker on the map
        Inputs:
        _______________
        - df: DataFrame
            dataset as a dataframe 
        - location_col: str
            location column within dataset df (e.g. city)
        - name_col: str
            names column within dataset df 
        - x: str
           location iterator (in case of run this func with df.apply)
        Outputs:
        ______________
        set of int and str for aggregations from main dataset df
    """
    new_df = new_df_beach[[location_col, name_col]].groupby([location_col,name_col]).count()
    nme_lst = []
    for item in new_df.index:
        if item[0] == x:
            nme_lst.append(item[1])
    
    #'Country : {0} \n Num of resources : {1} \n Names : {2}'.format([x],[df[location_col].value_counts()[x]],nme_lst)
    return 'Country : {0} \n Names : {1}'.format([x] ,nme_lst)

## Creat a func to count number of resources per country
def poptext_1(df,location_col,name_col, x):
    """
    Supporting func to show info for each marker on the map
        Inputs:
        _______________
        - df: DataFrame
            dataset as a dataframe 
        - location_col: str
            location column within dataset df (e.g. city)
        - name_col: str
            names column within dataset df 
        - x: str
           location iterator (in case of run this func with df.apply)
        Outputs:
        ______________
        set of int and str for aggregations from main dataset df
    """
    #info_df = pd.DataFrame()
    new_df = new_df_beach[[location_col, name_col]].groupby([location_col,name_col]).count()
    nme_lst = []
    for item in new_df.index:
        if item[0] == x:
            nme_lst.append(item[1])
            
    return 'Country : {0} \n Num of resources : {1} \n Names : {2}'.format([x],[df[location_col].value_counts()[x]],nme_lst)

## Creat a map 
## creating latitude/longtitude func
def PlotPoint(df, location_col, name_col):
    """
    Plotting coordinations points an a map
    Inputs:
    _______________
      - df: dataframe 
          the inputs dataset
      - location_col: str
          location column within dataset df (e.g. city)
      - name_col: str
          names column within dataset
    Outputs:
    _______________
        Geomap specifies team location 
    """   
    # converting location to a coordinate equivalents
    
    geolocator = Nominatim()
    Lat = list(itertools.chain(df[location_col].apply(lambda x: geolocator.geocode(x, timeout=15).latitude)))
    Long = list(itertools.chain(df[location_col].apply(lambda x: [geolocator.geocode(x, timeout=15).longitude])))
    location = np.mean(Lat), np.mean(Long)
    ResMap = folium.Map(location=location, zoom_start=7)
    LongLat = df[location_col].apply(
    lambda x: folium.Marker(
        location=[geolocator.geocode(x, timeout=15).latitude, geolocator.geocode(x, timeout=15).longitude],
        popup=folium.Popup(poptext_1(df, location_col,name_col, x), parse_html=True), 
        icon=folium.Icon(color='red',icon='info-sign', angle=10)).add_to(ResMap)
    if x == 'Stockholm' else folium.Marker(
        location=[geolocator.geocode(x, timeout=15).latitude, geolocator.geocode(x, timeout=15).longitude],
        popup=folium.Popup(poptext_1(df, location_col,name_col, x), parse_html=True), 
        icon=folium.Icon(icon='info-sign')).add_to(ResMap)
    )

    #Set the zoom to the maximum possible
    ResMap.fit_bounds(ResMap.get_bounds())
    return ResMap


    # Plotting coordinations on the map
    #folium.CircleMarker(location=[geolocator.geocode(x, timeout=15).latitude, geolocator.geocode(x, timeout=15).longitude]).add_to(ResMap)

In [29]:

#PlotPoint(new_df_beach, 'Location', 'Name')


## Appex

In [30]:
#df_location = pd.DataFrame(list(new_df_beach.groupby(['Location']).Location))
#df_location.set_index(df_location[0], inplace=True)
#np.array(list(zip(*new_df_beach.groupby(['Location', 'Name']).Location)))[0]
#new_df_beach['Location_eql_sthlm'] = np.where(new_df_beach['Location']=="Stockholm", True, False)
## setting up map to the center:
    # 1- location = df['Latitude'].mean(), df['Longitude'].mean()
    # 2- locationlist = df[["Latitude","Longitude"]].values.tolist()
    # 3- labels = df["xxx"].values.tolist()
    # 4- m = folium.Map(location=location, zoom_start=14)
    # 5- for point in range(len(locationlist)):
        #popup = folium.Popup(labels[point], parse_html=True)
        #folium.Marker(locationlist[point], popup=popup).add_to(m)
 

In [31]:
############################ Plotting using Plotly & Dash ##########################################
## Running credentials 

plotly.tools.set_credentials_file(username='omar.hazim', api_key='UnoUlwMh8vwec5ezDXzB')
mapbox_access_token = 'pk.eyJ1Ijoib21hcmhhemltIiwiYSI6ImNqY2s5cHk3MzNyZDEycm1tanV6c3pzdGUifQ.hLOK6z98WohsI19MmNBiHw'

## Converting location colunms to a cooredinations figures
geolocator = Nominatim()
Lat = list(itertools.chain(new_df_beach['Location'].apply(lambda x: geolocator.geocode(x, timeout=15).latitude)))
Long = list(itertools.chain(new_df_beach['Location'].apply(lambda x: [geolocator.geocode(x, timeout=15).longitude])))
cities = list(itertools.chain(new_df_beach['Location'].values))

info_1 = new_df_beach['Location'].apply(lambda i: poptext_2(new_df_beach, 'Location', 'Name', i))

## Creats Traces 
trace_1 = Scattermapbox(
        lat=Lat,
        lon=Long,
        mode='markers',
        marker = Marker(
            size = 13,
            opacity = .8,
            symbol = "circle",
        ),
       text = info_1,
)

## Map all coordinations
data_beach = Data([trace_1])

layout_beach = Layout(
    title = "Team locations - STHLM office",
    margin = dict(l = 20, r = 20, t = 60, b =20),
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
        lat=np.mean(Lat),
        lon=np.mean(Long)
        ),
        pitch=0,
        zoom=0.5,
    ),
)
fig_beach = dict(data=data_beach, layout=layout_beach)
py.iplot(fig_beach, filename='Beach Resources Mapbox')
    



In [32]:
#headers = {'user-agent':'Jupyter Dashboard Deployer'}
#files = {'file' : open('./Beach_work_WebTool.ipynb', 'r')}
#r = requests.post('http://127.0.0.1:3000/_api/notebooks/team', files=files)
#json.loads(r.content.decode())

In [33]:
#webbrowser.open('http://127.0.0.1:3000')

In [34]:

data = [ dict(
        type = 'choropleth',
        locations = country_conversion(cities),
        z = info_1,
        #text = ,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],
                      [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
        )
)]

layout = dict(
    title = '2014 Global GDP<br>Source:\
            <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='BEach-world-map' )

In [35]:
[country_conversion(set(cities)),set(cities)]


[['GBR',
  'SWE',
  'ESP',
  'Unknown code',
  'Unknown code',
  'FIN',
  'DEU',
  'DNK',
  'ITA',
  'Unknown code',
  'FRA'],
 {'Denmark',
  'Finland',
  'France',
  'Germany',
  'Italy',
  'Russia',
  'Spain',
  'Sweden',
  'United Kingdom',
  'united states',
  'united states '}]

In [38]:

country_conversion(['United States'])

['USA']